Derived from https://arxiv.org/pdf/1610.03022.pdf, 
1. No conv(3x3)/2 used 
2. Added an extra dense layer of 256 units

83.8% on val set

In [1]:
import warnings
warnings.filterwarnings("ignore")

import sys
import os
import tensorflow as tf

# sys.path.append("../libs")
sys.path.insert(1, '../')

from libs import input_data
from libs import models
from libs import trainer
from libs import freeze

W0119 11:27:09.904993  7404 deprecation_wrapper.py:119] From ..\libs\trainer.py:13: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0119 11:27:09.912265  7404 deprecation_wrapper.py:119] From ..\libs\trainer.py:13: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.



In [2]:
flags=tf.app.flags

In [3]:
flags=tf.app.flags
#Important Directories
flags.DEFINE_string('data_dir','..\\..\\_inputs\\raw','Train Data Folder')
flags.DEFINE_string('summaries_dir','..\\..\\summaries','Summaries Folder')
flags.DEFINE_string('train_dir','..\\..\\logs&checkpoint','Directory to write event logs and checkpoint')
flags.DEFINE_string('models_dir','..\\..\\models','Models Folder')
#Task Specific Parameters
flags.DEFINE_string('wanted_words','bed,bird,cat,dog,down,eight,five,four,go,happy,house,left,marvin,nine,no,off,on,one,right,seven,sheila,six,stop,three,tree,two,up,wow,yes,zero','Wanted Words')
flags.DEFINE_float('validation_percentage',10,'Validation Percentage')
flags.DEFINE_float('testing_percentage',10,'Testing Percentage')
flags.DEFINE_integer('sample_rate',16000,'Sample Rate')
flags.DEFINE_integer('clip_duration_ms',1000,'Clip Duration in ms')
flags.DEFINE_float('window_size_ms',30,'How long each spectogram timeslice is')
flags.DEFINE_float('window_stride_ms',10.0,'How far to move in time between frequency windows.')
flags.DEFINE_integer('dct_coefficient_count',40,'How many bins to use for the MFCC')
flags.DEFINE_float('time_shift_ms',100.0,'Range to randomly shift the training audio by in time.')

FLAGS=flags.FLAGS

In [4]:
model_architecture='convlstm'
start_checkpoint=None
logging_interval=1
eval_step_interval=1000
save_step_interval=1
silence_percentage=10.0
unknown_percentage=10.0
background_frequency=0.8
background_volume=0.3
learning_rate='0.0005,0.0001,0.00002,0.0001,0.00002' #Always seperated by comma, trains with each of the learning rate for the given number of iterations
train_steps='1000,1000,1000,1000,1000' #Declare  the training steps for which the learning rates will be used
batch_size=256

In [5]:
remaining_args = FLAGS([sys.argv[0]] + [flag for flag in sys.argv if flag.startswith("--")])
assert(remaining_args == [sys.argv[0]])

In [6]:
train_dir=os.path.join(FLAGS.data_dir,'train','audio')

In [7]:
model_settings = models.prepare_model_settings(
      len(input_data.prepare_words_list(FLAGS.wanted_words.split(','))),
      FLAGS.sample_rate, FLAGS.clip_duration_ms, FLAGS.window_size_ms,
      FLAGS.window_stride_ms, FLAGS.dct_coefficient_count)
audio_processor = input_data.AudioProcessor(
      train_dir, silence_percentage, unknown_percentage,
      FLAGS.wanted_words.split(','), FLAGS.validation_percentage,
      FLAGS.testing_percentage, model_settings)

..\..\_inputs\raw\train\audio\*\*.wav


W0119 11:27:34.901259  7404 deprecation_wrapper.py:119] From ..\libs\input_data.py:305: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0119 11:27:34.932041  7404 deprecation_wrapper.py:119] From ..\libs\input_data.py:306: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



..\..\_inputs\raw\train\audio\_background_noise_\doing_the_dishes.wav
..\..\_inputs\raw\train\audio\_background_noise_\dude_miaowing.wav
..\..\_inputs\raw\train\audio\_background_noise_\exercise_bike.wav
..\..\_inputs\raw\train\audio\_background_noise_\pink_noise.wav
..\..\_inputs\raw\train\audio\_background_noise_\running_tap.wav
..\..\_inputs\raw\train\audio\_background_noise_\white_noise.wav
Tensor("AudioSpectrogram:0", shape=(?, 98, 257), dtype=float32)


In [ ]:
def get_train_data(args):
    sess=args
    time_shift_samples = int((FLAGS.time_shift_ms * FLAGS.sample_rate) / 1000)
    train_data, train_ground_truth = audio_processor.get_data(
        batch_size, 0, model_settings, background_frequency,
        background_volume, time_shift_samples, 'training', sess)
    return train_data, train_ground_truth

def get_val_data(args):
    '''
    Input: (sess,offset)
    '''
    sess,i=args
    validation_data, validation_ground_truth = (
        audio_processor.get_data(batch_size, i, model_settings, 0.0,
                                     0.0, 0, 'validation', sess))
    return validation_data,validation_ground_truth

In [10]:
def main(_):
    sess=tf.InteractiveSession()
    # Placeholders
    data_size = model_settings['data_size']
    label_count = model_settings['label_count']
    data_input = tf.placeholder(
      tf.float32, [None, data_size], name='data_input')
    ground_truth_input = tf.placeholder(
      tf.float32, [None, label_count], name='groundtruth_input')
    set_size = audio_processor.set_size('validation')
    label_count = model_settings['label_count']
    # Create Model
    
    logits, dropout_prob = models.create_model( 
      data_input,
      model_settings,
      model_architecture,
      is_training=True)
    #Start Training
    extra_args=(dropout_prob,label_count,batch_size,set_size)
    trainer.train(sess,logits,data_input,ground_truth_input,get_train_data,
                  get_val_data,train_steps,learning_rate,eval_step_interval, logging_interval=logging_interval,
                  start_checkpoint=start_checkpoint,checkpoint_interval=save_step_interval,
                  model_name=model_architecture,train_dir=FLAGS.train_dir,
                  summaries_dir=FLAGS.summaries_dir,args=extra_args,dropout=0.8)

In [11]:
tf.app.run(main=main)

W0119 11:28:40.765048  7404 deprecation_wrapper.py:119] From ..\libs\models.py:382: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0119 11:28:40.902948  7404 deprecation.py:323] From ..\libs\models.py:392: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
W0119 11:28:42.057269  7404 deprecation.py:506] From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0119 11:28:42.097864  7404 deprecation.py:506] From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\contrib\rnn\python\ops\rnn_cell.py:2242: calling Consta

I0119 11:37:57.992708  7404 trainer.py:126] Step #15: rate 0.000500, accuracy 9.8%, cross entropy 3.347271
I0119 11:37:57.992708  7404 trainer.py:172] Saving to "..\..\logs&checkpoint\convlstm\ckpt-15"
I0119 11:38:32.606708  7404 trainer.py:126] Step #16: rate 0.000500, accuracy 5.9%, cross entropy 3.403572
I0119 11:38:32.625792  7404 trainer.py:172] Saving to "..\..\logs&checkpoint\convlstm\ckpt-16"
I0119 11:39:09.140266  7404 trainer.py:126] Step #17: rate 0.000500, accuracy 5.1%, cross entropy 3.396159
I0119 11:39:09.140266  7404 trainer.py:172] Saving to "..\..\logs&checkpoint\convlstm\ckpt-17"
I0119 11:39:46.067090  7404 trainer.py:126] Step #18: rate 0.000500, accuracy 10.2%, cross entropy 3.329290
I0119 11:39:46.069243  7404 trainer.py:172] Saving to "..\..\logs&checkpoint\convlstm\ckpt-18"
I0119 11:40:19.203418  7404 trainer.py:126] Step #19: rate 0.000500, accuracy 9.0%, cross entropy 3.303377
I0119 11:40:19.219246  7404 trainer.py:172] Saving to "..\..\logs&checkpoint\convlst

I0119 12:02:39.586322  7404 trainer.py:172] Saving to "..\..\logs&checkpoint\convlstm\ckpt-55"
I0119 12:03:10.039048  7404 trainer.py:126] Step #56: rate 0.000500, accuracy 8.6%, cross entropy 3.309094
I0119 12:03:10.057994  7404 trainer.py:172] Saving to "..\..\logs&checkpoint\convlstm\ckpt-56"
I0119 12:03:39.939132  7404 trainer.py:126] Step #57: rate 0.000500, accuracy 9.4%, cross entropy 3.256677
I0119 12:03:39.945125  7404 trainer.py:172] Saving to "..\..\logs&checkpoint\convlstm\ckpt-57"
I0119 12:04:11.563002  7404 trainer.py:126] Step #58: rate 0.000500, accuracy 9.8%, cross entropy 3.272672
I0119 12:04:11.566992  7404 trainer.py:172] Saving to "..\..\logs&checkpoint\convlstm\ckpt-58"
I0119 12:04:40.749057  7404 trainer.py:126] Step #59: rate 0.000500, accuracy 11.7%, cross entropy 3.311033
I0119 12:04:40.756037  7404 trainer.py:172] Saving to "..\..\logs&checkpoint\convlstm\ckpt-59"
I0119 12:05:11.996434  7404 trainer.py:126] Step #60: rate 0.000500, accuracy 15.6%, cross entro

I0119 12:23:06.664076  7404 trainer.py:126] Step #96: rate 0.000500, accuracy 15.2%, cross entropy 3.115656
I0119 12:23:06.666070  7404 trainer.py:172] Saving to "..\..\logs&checkpoint\convlstm\ckpt-96"
I0119 12:23:36.572533  7404 trainer.py:126] Step #97: rate 0.000500, accuracy 12.1%, cross entropy 3.185957
I0119 12:23:36.573530  7404 trainer.py:172] Saving to "..\..\logs&checkpoint\convlstm\ckpt-97"
I0119 12:24:06.114663  7404 trainer.py:126] Step #98: rate 0.000500, accuracy 14.1%, cross entropy 3.151109
I0119 12:24:06.118650  7404 trainer.py:172] Saving to "..\..\logs&checkpoint\convlstm\ckpt-98"
I0119 12:24:35.802648  7404 trainer.py:126] Step #99: rate 0.000500, accuracy 16.8%, cross entropy 3.065239
I0119 12:24:35.808631  7404 trainer.py:172] Saving to "..\..\logs&checkpoint\convlstm\ckpt-99"
I0119 12:25:04.709764  7404 trainer.py:126] Step #100: rate 0.000500, accuracy 13.3%, cross entropy 3.146887
I0119 12:25:04.717741  7404 trainer.py:172] Saving to "..\..\logs&checkpoint\co

I0119 12:42:28.362116  7404 trainer.py:172] Saving to "..\..\logs&checkpoint\convlstm\ckpt-136"
I0119 12:42:58.273192  7404 trainer.py:126] Step #137: rate 0.000500, accuracy 16.0%, cross entropy 2.983772
I0119 12:42:58.274193  7404 trainer.py:172] Saving to "..\..\logs&checkpoint\convlstm\ckpt-137"
I0119 12:43:29.426110  7404 trainer.py:126] Step #138: rate 0.000500, accuracy 18.0%, cross entropy 2.892891
I0119 12:43:29.428105  7404 trainer.py:172] Saving to "..\..\logs&checkpoint\convlstm\ckpt-138"
I0119 12:44:01.509117  7404 trainer.py:126] Step #139: rate 0.000500, accuracy 16.8%, cross entropy 3.046561
I0119 12:44:01.512108  7404 trainer.py:172] Saving to "..\..\logs&checkpoint\convlstm\ckpt-139"
I0119 12:44:32.165011  7404 trainer.py:126] Step #140: rate 0.000500, accuracy 19.1%, cross entropy 2.868938
I0119 12:44:32.178975  7404 trainer.py:172] Saving to "..\..\logs&checkpoint\convlstm\ckpt-140"
I0119 12:45:02.638271  7404 trainer.py:126] Step #141: rate 0.000500, accuracy 21.5%

...

I0119 14:10:55.064155  7404 trainer.py:172] Saving to "..\..\logs&checkpoint\convlstm\ckpt-296"
I0119 14:11:15.421370  7404 trainer.py:126] Step #297: rate 0.000500, accuracy 40.6%, cross entropy 2.002370
I0119 14:11:15.421370  7404 trainer.py:172] Saving to "..\..\logs&checkpoint\convlstm\ckpt-297"
I0119 14:11:35.156213  7404 trainer.py:126] Step #298: rate 0.000500, accuracy 39.8%, cross entropy 1.994131
I0119 14:11:35.156213  7404 trainer.py:172] Saving to "..\..\logs&checkpoint\convlstm\ckpt-298"
I0119 14:11:55.678409  7404 trainer.py:126] Step #299: rate 0.000500, accuracy 41.4%, cross entropy 1.962455
I0119 14:11:55.678409  7404 trainer.py:172] Saving to "..\..\logs&checkpoint\convlstm\ckpt-299"
I0119 14:12:14.743406  7404 trainer.py:126] Step #300: rate 0.000500, accuracy 41.4%, cross entropy 1.966854
I0119 14:12:14.743406  7404 trainer.py:172] Saving to "..\..\logs&checkpoint\convlstm\ckpt-300"
I0119 14:12:36.131905  7404 trainer.py:126] Step #301: rate 0.000500, accuracy 37.9%

I0119 14:24:12.408719  7404 trainer.py:172] Saving to "..\..\logs&checkpoint\convlstm\ckpt-336"
I0119 14:24:32.690819  7404 trainer.py:126] Step #337: rate 0.000500, accuracy 42.6%, cross entropy 1.953153
I0119 14:24:32.690819  7404 trainer.py:172] Saving to "..\..\logs&checkpoint\convlstm\ckpt-337"
I0119 14:24:51.549912  7404 trainer.py:126] Step #338: rate 0.000500, accuracy 48.0%, cross entropy 1.670931
I0119 14:24:51.565535  7404 trainer.py:172] Saving to "..\..\logs&checkpoint\convlstm\ckpt-338"
I0119 14:25:12.319007  7404 trainer.py:126] Step #339: rate 0.000500, accuracy 42.6%, cross entropy 1.854600
I0119 14:25:12.319007  7404 trainer.py:172] Saving to "..\..\logs&checkpoint\convlstm\ckpt-339"
I0119 14:25:32.956942  7404 trainer.py:126] Step #340: rate 0.000500, accuracy 45.7%, cross entropy 1.965050
I0119 14:25:32.961929  7404 trainer.py:172] Saving to "..\..\logs&checkpoint\convlstm\ckpt-340"
I0119 14:25:52.490569  7404 trainer.py:126] Step #341: rate 0.000500, accuracy 44.5%

I0119 14:37:42.639094  7404 trainer.py:172] Saving to "..\..\logs&checkpoint\convlstm\ckpt-376"
I0119 14:38:02.631954  7404 trainer.py:126] Step #377: rate 0.000500, accuracy 49.6%, cross entropy 1.690868
I0119 14:38:02.631954  7404 trainer.py:172] Saving to "..\..\logs&checkpoint\convlstm\ckpt-377"
I0119 14:38:22.710602  7404 trainer.py:126] Step #378: rate 0.000500, accuracy 52.3%, cross entropy 1.615850
I0119 14:38:22.710602  7404 trainer.py:172] Saving to "..\..\logs&checkpoint\convlstm\ckpt-378"
I0119 14:38:44.122359  7404 trainer.py:126] Step #379: rate 0.000500, accuracy 46.9%, cross entropy 1.657530
I0119 14:38:44.123358  7404 trainer.py:172] Saving to "..\..\logs&checkpoint\convlstm\ckpt-379"
I0119 14:39:03.326200  7404 trainer.py:126] Step #380: rate 0.000500, accuracy 53.1%, cross entropy 1.476275
I0119 14:39:03.341813  7404 trainer.py:172] Saving to "..\..\logs&checkpoint\convlstm\ckpt-380"
I0119 14:39:23.515283  7404 trainer.py:126] Step #381: rate 0.000500, accuracy 51.6%

I0119 14:51:06.109616  7404 trainer.py:172] Saving to "..\..\logs&checkpoint\convlstm\ckpt-416"
I0119 14:51:26.018729  7404 trainer.py:126] Step #417: rate 0.000500, accuracy 52.7%, cross entropy 1.667759
I0119 14:51:26.018729  7404 trainer.py:172] Saving to "..\..\logs&checkpoint\convlstm\ckpt-417"
I0119 14:51:45.910102  7404 trainer.py:126] Step #418: rate 0.000500, accuracy 53.1%, cross entropy 1.593780
I0119 14:51:45.910102  7404 trainer.py:172] Saving to "..\..\logs&checkpoint\convlstm\ckpt-418"
I0119 14:52:24.529219  7404 trainer.py:126] Step #419: rate 0.000500, accuracy 56.2%, cross entropy 1.561603
I0119 14:52:24.529219  7404 trainer.py:172] Saving to "..\..\logs&checkpoint\convlstm\ckpt-419"
I0119 14:53:02.248051  7404 trainer.py:126] Step #420: rate 0.000500, accuracy 58.2%, cross entropy 1.438595
I0119 14:53:02.263361  7404 trainer.py:172] Saving to "..\..\logs&checkpoint\convlstm\ckpt-420"
I0119 14:53:39.951872  7404 trainer.py:126] Step #421: rate 0.000500, accuracy 54.3%

KeyboardInterrupt: 

In [ ]:
# save_checkpoint='../logs&checkpoint/convlstm/ckpt-52000'

In [ ]:
# save_path=os.path.join(FLAGS.models_dir,model_architecture,'%s.pb'%os.path.basename(save_checkpoint))
# freeze.freeze_graph(FLAGS,model_architecture,save_checkpoint,save_path)

In [ ]:
# save_path=os.path.join(FLAGS.models_dir,model_architecture,'%s-batched.pb'%os.path.basename(save_checkpoint))
# freeze.freeze_graph(FLAGS,model_architecture,save_checkpoint,save_path,batched=True)